# 🧩 Chapter 6: Precision 3D - Part 2: Registration

In this second part of Chapter 6, we tackle **Point Cloud Registration**. This is the process of aligning two point clouds into a single coordinate system. It is fundamental for stitching together 3D scans.

**Objectives:**
1.  **Global Registration**: Aligning clouds without prior knowledge using RANSAC and FPFH features.
2.  **Local Registration**: Refining the alignment using the Iterative Closest Point (ICP) algorithm.

In [ ]:
import numpy as np
import open3d as o3d
import copy
import time

## 1. Load Source and Target Clouds

We have a "Source" cloud (to be moved) and a "Target" cloud (fixed).

In [ ]:
# Load data
source_path = "../DATA/registration_source.ply"
target_path = "../DATA/global_todo_registration_target.ply"

source = o3d.io.read_point_cloud(source_path)
target = o3d.io.read_point_cloud(target_path)

# Visualize initial state
print("Initial State: Source is NOT aligned with Target.")
source.paint_uniform_color([1, 0.7, 0]) # Yellow Source
target.paint_uniform_color([0, 0.6, 0.9]) # Blue Target
o3d.visualization.draw_geometries([source, target])

## 2. Feature Extraction (FPFH)

Before we can align them globally, we need to describe the geometry around each point using **Fast Point Feature Histograms (FPFH)**. This allows us to find matching points even if they are far apart.

We also downsample the point clouds to speed up the process.

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    # 1. Downsample
    pcd_down = pcd.voxel_down_sample(voxel_size)
    
    # 2. Estimate Normals (Required for FPFH)
    radius_normal = voxel_size * 2.0
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
    # 3. Compute FPFH Features
    radius_feature = voxel_size * 5.0
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    
    return pcd_down, pcd_fpfh

voxel_size = 1.5 # Coarse voxel size for Global Registration
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)

print(f"Source Downsampled: {len(source_down.points)} points.")
print(f"Target Downsampled: {len(target_down.points)} points.")

## 3. Global Registration (RANSAC)

We use **RANSAC (Random Sample Consensus)** to find match correspondences between our FPFH features. This gives us a rough match.

In [ ]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, # Match 3 points at least
        [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )
    return result

start = time.time()
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(f"Global Registration took {time.time() - start:.2f} sec.")
print("Transformation Matrix:\n", result_ransac.transformation)

# Apply Transformation
source.transform(result_ransac.transformation)
o3d.visualization.draw_geometries([source, target], window_name="After Global Registration")

## 4. Local Refinement (ICP)

Now that the clouds are roughly aligned, we use **Iterative Closest Point (ICP)** to perfect the match. We use a smaller threshold here.

In [ ]:
# Ensure normals are estimated for the original dense clouds
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size, max_nn=30))
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size, max_nn=30))

# Threshold for ICP (Usually smaller than global)
icp_threshold = voxel_size * 0.4

start = time.time()
result_icp = o3d.pipelines.registration.registration_icp(
    source, target, icp_threshold, np.identity(4), # Source is already transformed, so initial guess is Identity
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print(f"ICP Refinement took {time.time() - start:.2f} sec.")
print("Refined Transformation:\n", result_icp.transformation)

# Apply Final Transformation (Note: this adds to the previous one)
source.transform(result_icp.transformation)
source.paint_uniform_color([1, 0, 0]) # Red for final alignment
o3d.visualization.draw_geometries([source, target], window_name="Final ICP Registration")